In [38]:
import pytraj as pt
import numpy as np
import pandas as pd
import tqdm
import sys
import gc
import copy
import nglview as nv

import ipywidgets as widgets
from ipywidgets import interact, interact_manual


In [3]:
strucDir='structures/'
viewPDBname='Piezo_PIP2.prot_only.pdb'
trajName='Piezo_PIP2.prot_only.dcd'
topName='Piezo_PIP2.prot_only.parm7'
viewStruc=pt.load(strucDir+'Piezo_PIP2.prot_only.pdb')
iterTraj=pt.iterload(strucDir+trajName,
                     top=strucDir+topName)

print viewStruc
print iterTraj

pytraj.Trajectory, 1 frames: 
Size: 0.000000 (GB)
<Topology: 70296 atoms, 4254 residues, 12 mols, PBC with box type = ortho>
           
pytraj.TrajectoryIterator, 1592 frames: 
Size: 2.000000 (GB)
<Topology: 70296 atoms, 4254 residues, 12 mols, PBC with box type = ortho>
           


NGLView can allow us to look at the structure directly within the notebook.

Warning: google colab seems to have trouble with this as nglview apparently crashes the kernel

In [4]:
color_scheme=[
    ['blue','ARG LYS HIS HIP HID HIE HSD'],
    ['red','ASP GLU'],
    ['green','SER THR ASN GLN'],
    ['white','CYS SEC GLY PRO ALA VAL ILE LEU MET PHE TYR TRP']
]

In [68]:
view=nv.show_pytraj(viewStruc)
view.clear_representations()
view.add_representation('cartoon',color='white',alpha=.5)
view.add_representation('licorice',
                        color='blue',
                        selection='HSD LYS ARG')
view

NGLWidget()

Lets inspect the residues. A widget proves helpful here since there are many and we 
will need to be able to look up information about the atoms in each one.

In [50]:
@interact
def get_residue_atomInfo(Selected_Residue=[
    "%s_%g"%(iterTraj.top.residue(resID).name,
             iterTraj.top.residue(resID).index) \
    for resID in np.arange(iterTraj.top.n_residues)]):
    resNum=int(Selected_Residue.split('_')[1])+1
    atomDat=np.array(map(lambda x: x.replace('#','').split(),
        pt.resinfo(iterTraj.top,'atominfo :%g'%resNum,viewStruc[0],task='atominfo').split('\n')))
    return pd.DataFrame(data=list(atomDat[1:-1]),columns=atomDat[0])

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnU2VsZWN0ZWRfUmVzaWR1ZScsIG9wdGlvbnM9KHUnQVJHXzAnLCB1J0xFVV8xJywgdSdMRVVfMicsIHUnQVNQXzPigKY=


In [51]:
from pytraj.utils.progress import ProgressBarTrajectory

We now have the information we need to compute SASA data for each cationic residue (HSD, LYS, ARG).

We will only include atoms of the cationic regions.

This will yield a SASA calculation for each frame. The data table will be quite large.

We will process this raw data during the analysis step.

In [58]:
basicResidueIDs=[res.original_resid for res in \
                 viewStruc.top.residues if res.name in ['HSD','LYS','ARG']]

masksDict={
    'ARG':'@NE,NH1,NH2,HE,HH11,HH12,HH21,HH22,CZ',
    'LYS':'@NZ,HZ1,HZ2,HZ3',
    'HSD':'@CG,CE1,CE2,CD2,ND1,NE2,HD1,HD2',
}
probeOffset=2.15
commandList=['surf :%g%s offset %.2f'%(
    iterTraj.top.residue(resID-1).original_resid,
    masksDict[iterTraj.top.residue(resID-1).name],
    probeOffset) for resID in basicResidueIDs]
pbar=ProgressBarTrajectory(iterTraj,style='tqdm')
SASA_data=pt.compute(commandList,pbar)

In [59]:
basicResidueSASAtable=pd.DataFrame(np.arange(iterTraj.n_frames),columns=['Frame'])
basicResidueSASAtable['System']='Piezo_PIP2'
for ii,resID in tqdm.tqdm_notebook(enumerate(basicResidueIDs)):
    entryName='%s_%g'%(iterTraj.top.residue(resID-1).name,resID)
    basicResidueSASAtable[entryName]=SASA_data.values()[ii]
basicResidueSASAtable=basicResidueSASAtable[np.concatenate([
    ['System','Frame'],
    basicResidueSASAtable.columns[2:]])]
basicResidueSASAtable.to_csv('basicResidueSASA.csv',index=False)
basicResidueSASAtable.head()

,System,Frame,ARG_1,ARG_15,ARG_21,ARG_22,HSD_27,LYS_30,LYS_42,ARG_63,...,HSD_4198,ARG_4210,LYS_4213,ARG_4222,ARG_4225,LYS_4236,ARG_4242,LYS_4249,ARG_4252,ARG_4254
0,Piezo_PIP2,0,287.440560,274.898960,324.281827,540.105388,382.686118,172.673159,102.452724,221.306778,...,494.523318,278.618895,116.343258,629.556884,299.282841,140.083915,646.955674,103.634013,282.838529,205.066848
1,Piezo_PIP2,1,334.816766,183.629963,283.576915,578.829623,384.278017,103.769143,109.288093,243.110699,...,355.490064,257.677793,98.921705,453.885378,416.310184,131.922552,462.089912,95.179939,242.496894,223.123915
2,Piezo_PIP2,2,275.073755,202.255450,342.219028,560.460974,336.459908,111.187833,102.056689,240.717753,...,403.091530,300.392173,85.993276,465.202440,257.272547,72.081927,535.732493,82.812868,218.384865,218.979063
3,Piezo_PIP2,3,290.577028,193.342330,351.953645,680.849566,356.348260,126.432774,103.091409,239.646221,...,418.481454,332.128525,96.114586,567.150851,259.985159,88.943817,501.348161,95.815112,247.253744,220.677821
4,Piezo_PIP2,4,233.699040,203.015984,240.717316,603.387966,328.979551,127.112652,111.054757,232.711257,...,410.296426,309.629827,104.504957,468.422037,291.298963,93.244969,460.571667,93.286728,271.298287,222.949990
